In [ ]:
# default_exp sampler.sample_tree

# Sample Tree

> A tree data structure to count circuit and subset selection in protocol runs

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
#export
import qsam.math as math

from anytree import RenderTree, NodeMixin
from anytree.exporter import JsonExporter
from anytree.importer import JsonImporter, DictImporter

For subset sampling we need a new data structure to keep track of the circuit and circuit-subsets we are sampling from per protocol run. The natural structure for this problem is a Tree which we call `SampleTree`, populated with so-called `CountNodes`:

In [ ]:
#export
class CountNode(NodeMixin):
    def __init__(self, name, parent=None, children=None, counts=0, 
                 ckey=None, is_deterministic=False, is_fail=False):
        super(CountNode, self).__init__()
        
        self.name = name
        self.ckey = ckey
        self.counts = int(counts)
        self.is_deterministic = is_deterministic
        self.is_fail = is_fail
        
        self.parent = parent
        if children:
            self.children = children
            
    @property
    def rate(self):
        assert not self.ckey and not self.is_root
        if self.parent.counts == 0: return 0
        else: return self.counts / self.parent.counts

    @property
    def var(self, var_fn=math.Wilson_var):
        assert not self.ckey and not self.is_root
        if self.parent.counts == 0: return 1.0
        else: return var_fn(self.rate, self.parent.counts)
    
    def __str__(self):
        if self.is_root or self.ckey:
            return f'{self.name} ({self.counts})'
        else:
            return f'{self.name} ({self.counts}/{self.parent.counts}, {self.var:.2e})'

In [ ]:
#export
class SampleTree:
    def __init__(self, root=None):
        self.root = root
    
    def add(self, name, parent=None, **kwargs):
        if parent == None:
            if self.root == None:
                self.root = CountNode(name, **kwargs)
            return self.root
        else:
            child_match = [n for n in parent.children if n.name==name]
            if child_match: return child_match[0]
            else: return CountNode(name, parent=parent, **kwargs)
        
    def update(self, name, parent=None, **kwargs):
        node = self.add(name, parent, **kwargs)
        node.counts += 1
        return node
        
    def detach(self, node):
        node.parent = None
        
    def rate(self, const):
        """Calculate logical failure rate
        Sum of products of constants and rates along a path
        for every path leading to a fail.
        """
        p = 0
        for leaf in self.root.leaves:
            if not leaf.is_root and not leaf.parent.is_deterministic and leaf.is_fail: 
                prod = 1
                for n in leaf.path[1:]:
                    if n.ckey: prod *= const[n.ckey[0]][n.ckey[1]]
                    else: prod *= n.rate
                p += prod
        return p
    
    def delta(self, const):
        """Calculate the cutoff error (delta)
        Sum of products of constants and rates along a path
        for every path that has been sampled.
        """
        delta = 1.0
        for leaf in self.root.leaves:
            if not leaf.is_root:
                prod = 1.0
                for n in leaf.path[1:]:
                    if n.ckey: prod *= const[n.ckey[0]][n.ckey[1]]
                    else: prod *= n.rate
                delta -= prod
        return delta

    def var(self, const):
        """Calculate variance of logical failure rate
        Apply Gaussian error propagation to every differentiable node.
        All non-constant nodes are differentiable, as they contain a rate.
        """
        var = 0
        for node in [n for n in self.root.descendants if not n.ckey]: # all circuit nodes incl. None and Fail.
            if not node.parent.is_deterministic:
                
                twig = 1.0
                for n in node.path[1:-1]:
                    if n.ckey: twig *= const[n.ckey[0]][n.ckey[1]]
                    else: twig *= n.rate 
                
                if node.is_leaf and len(node.siblings) == 0:
                    var += node.var * twig**2 # count single None leaf nodes
                else:
                    subtree = 0
                    for leaf in node.leaves:
                        if not leaf.is_fail:  # No-Fail paths in subtree multiply to 0.
                            continue
                        prod = 1
                        for n in leaf.iter_path_reverse():
                            if n == node: break
                            elif n.ckey: prod *= const[n.ckey[0]][n.ckey[1]]
                            else: prod *= n.rate                
                        subtree += prod
                    var += node.var * twig**2 * subtree**2
        return var
    
    def delta_var(self, const):
        """Calculate the variance of the cutoff error (delta)
        Apply Gaussian error propagation to every sampled differentiable node.
        Fail/No-Fail branching can be ignored as it always sums to 1 here.
        """
        var = 0
        for node in [n for n in self.root.descendants if not n.ckey and not n.is_leaf]:
            if not node.parent.is_deterministic:
                twig = 1.0
                for n in node.path[1:-1]:
                    if n.ckey: twig *= const[n.ckey[0]][n.ckey[1]]
                    else: twig *= n.rate

                subtree = 0
                for leaf in node.leaves:
                    prod = 1
                    for n in leaf.iter_path_reverse():
                        if n == node: break
                        elif n.ckey: prod *= const[n.ckey[0]][n.ckey[1]]
                        else: prod *= n.rate                
                    subtree += prod
                var += node.var * twig**2 * subtree**2
        return var
    
    def __str__(self):
        return '\n'.join([f'{pre}{node}' for pre, _, node in RenderTree(self.root)])
    
    def display(self, fname='temp.png'):
        
        try:
            from anytree.exporter import UniqueDotExporter
            from IPython.display import Image, display ## ! requirements: jupyter, graphviz
        except ImportError as e:
            print(f"Cannot display tree due to missing libraries. Original error message {e}")
            return None
        
        def edgeattrfunc(node, child):
            weight = 10.0 * child.counts / self.root.counts
            if weight > 5.0:
                weight = 5.0
            elif weight < 0.2:
                weight = 0.2
            return f'penwidth="{weight}"'

        def nodeattrfunc(node):
            node_form = ""
            if node.is_fail: 
                node_form = ', style=filled, fillcolor="#ff9999"'
            return f'label="{node.name}"' + node_form

        img = UniqueDotExporter(self.root, 
                          edgeattrfunc=edgeattrfunc,
                          nodeattrfunc=nodeattrfunc).to_picture(fname)
        display(Image(fname))
        return fname
    
    def save(self, fname):
        exporter = JsonExporter(indent=2, sort_keys=False, default=str)
        with open(fname, 'w') as f:
            exporter.write(self.root, f)

    def load(self, fname):
        dictimporter = DictImporter(CountNode)
        importer = JsonImporter(dictimporter)
        with open(fname, 'r') as f:
            self.root = importer.read(f)
        return self